In [1]:
! wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
import numpy as np 
import pandas as pd
import os 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection  import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

--2021-09-25 15:48:04--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘AB_NYC_2019.csv.1’

AB_NYC_2019.csv.1   100%[===================>]   6.75M  --.-KB/s    in 0.1s    

2021-09-25 15:48:04 (65.8 MB/s) - ‘AB_NYC_2019.csv.1’ saved [7077973/7077973]



In [2]:
data = pd.read_csv('AB_NYC_2019.csv')

In [3]:
df = data.loc[:, ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
                    'calculated_host_listings_count', 'availability_365']]

In [4]:
df.fillna(0, inplace=True)

In [5]:
df['neighbourhood_group'].value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [7]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [8]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [9]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

In [10]:
corr_matrix = df_train.corr()
corr_matrix

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [11]:
col = corr_matrix.columns
col = pd.Series.tolist(col)

max = 0
for i in range(len(col)):
  for j in range(len(col)):
    if(i > j):
      if(corr_matrix.loc[col[i]][col[j]]>max):
        max = corr_matrix.loc[col[i]][col[j]]
        r, c = col[i], col[j]

print(r, " ", c)

reviews_per_month   number_of_reviews


In [12]:
above_average_train = (y_train >= 152).astype(int)
above_average_val = (y_val >= 152).astype(int)
s1 = mutual_info_score(above_average_train, df_train.neighbourhood_group).round(2)
s2 = mutual_info_score(above_average_train, df_train.room_type).round(2)
print(s1, " ", s2)

0.05   0.14


In [13]:
df_train.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [14]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [15]:
X_train.shape

(29337, 15)

In [16]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)

In [17]:
model.fit(X_train, above_average_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
y_pred_val = model.predict(X_val)

In [19]:
score = accuracy_score(above_average_val, y_pred_val)
score.round(2)

0.79

In [20]:
def one_hot_encoding(df_train, df_val):
  dv = DictVectorizer(sparse=False)

  train_dict = df_train.to_dict(orient='records')
  X_train = dv.fit_transform(train_dict)

  val_dict = df_val.to_dict(orient='records')
  X_val = dv.transform(val_dict)
  return X_train, X_val

In [21]:
def accuracy_after_removing_feature(feature):
  X_train_new = df_train.drop(feature, axis = 1)
  X_val_new = df_val.drop(feature, axis = 1)
  X_train_new, X_val_new = one_hot_encoding(X_train_new, X_val_new)
  model.fit(X_train_new, above_average_train)
  y_pred_val_new = model.predict(X_val_new)
  score = accuracy_score(above_average_val, y_pred_val_new)
  return score

In [22]:
# removing features
dic_diff  = {}
for feature in list(df_train.columns):
  dic_diff[feature] = abs(accuracy_after_removing_feature(feature)-score)

dic_diff

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'availability_365': 0.004908477349422236,
 'calculated_host_listings_count': 0.0002045198895592737,
 'latitude': 0.00010225994477963685,
 'longitude': 0.00030677983433891054,
 'minimum_nights': 0.0009203395030166206,
 'neighbourhood_group': 0.03548420083853154,
 'number_of_reviews': 0.0005112997238981842,
 'reviews_per_month': 0.0014316392269148048,
 'room_type': 0.06994580222926683}

In [23]:
y_train = np.log1p(y_train)
y_train
scores = []

for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha)

    model.fit(X_train, y_train)

    scores.append(round(mean_squared_error(model.predict(X_val), y_val, squared=False), 3))

scores

[268.915, 268.914, 268.914, 268.914, 268.916]